#### For development


In this file, following lines are processed.
We distribute processed file to students.

**TASK DEFINITION : to predidct Churn(:= user has no record during D+30 ~ D+210 starting from account creation.), given records of first 30 days.**  
(*this task is used for samsung data programming advanced, 16th (lecture done in May 2020) *)

- 1. Eliminate posts except questions and answers
- 2. Extract the dataset for the period of the dataset: July 31, 2008 - July 31, 2012
- 3. remove users that have not enough records (i.e, July31,2012 - 6month < AccountCreationDate )
- 4. drop reputation column / creationdateofowner
- 5.1. Eliminate answers which do not have the question
- 5.2. If a answer is faster than question of it, eliminate the answer
- 6. verify churners
- 7. split data


In [1]:
import pandas as pd
import numpy as np

In [2]:
users_df = pd.read_pickle('users_reduce.pkl')
posts_df = pd.read_pickle('posts_reduce.pkl')

In [3]:
users_df.shape, posts_df.shape
print(users_df.info(), posts_df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 992110 entries, -1 to 11979186
Data columns (total 3 columns):
Reputation        992110 non-null int32
CreationDate      992110 non-null datetime64[ns]
LastAccessDate    992110 non-null datetime64[ns]
dtypes: datetime64[ns](2), int32(1)
memory usage: 26.5 MB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 11324326 entries, 4 to 14625187
Data columns (total 10 columns):
PostTypeId             int32
AcceptedAnswerId       float32
CreationDate           datetime64[ns]
Score                  float32
OwnerUserId            int32
AnswerCount            float32
CommentCount           float32
ParentId               float32
CreationDateOfOwner    datetime64[ns]
BodyWordNum            float64
dtypes: datetime64[ns](2), float32(5), float64(1), int32(2)
memory usage: 648.0 MB
None None


In [4]:
posts_df['PostId'] = posts_df.index

In [5]:
# Eliminate posts except questions and answers
posts_df = posts_df[(posts_df['PostTypeId'] == 1) | (posts_df['PostTypeId'] == 2)]

print(posts_df.shape)
print(posts_df.info())

(11291882, 11)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 11291882 entries, 4 to 14625187
Data columns (total 11 columns):
PostTypeId             int32
AcceptedAnswerId       float32
CreationDate           datetime64[ns]
Score                  float32
OwnerUserId            int32
AnswerCount            float32
CommentCount           float32
ParentId               float32
CreationDateOfOwner    datetime64[ns]
BodyWordNum            float64
PostId                 int64
dtypes: datetime64[ns](2), float32(5), float64(1), int32(2), int64(1)
memory usage: 732.3 MB
None


In [6]:
# 2 Extract the dataset for the period of the dataset: July 31, 2008 - July 31, 2012
start_time = pd.to_datetime('2008-07-31')
end_time = pd.to_datetime('2012-07-31')
end_time_2 = pd.to_datetime('2012-01-31')

In [7]:
posts_df = posts_df[(pd.to_datetime(posts_df['CreationDate']) >= start_time) & (pd.to_datetime(posts_df['CreationDate']) <= end_time)]
users_df = users_df[(pd.to_datetime(users_df['CreationDate']) >= start_time) & (pd.to_datetime(users_df['CreationDate']) <= end_time_2)]

print(posts_df.shape, users_df.shape)


(9499165, 11) (692028, 3)


In [8]:
# 3 remove users that have not enough records (i.e, July31,2012 - 6month < AccountCreationDate )

sixMonthsBeforeEnd = pd.to_datetime('2012-01-30')

users_df = users_df[pd.to_datetime(users_df['CreationDate']) < sixMonthsBeforeEnd]
print(posts_df.shape, users_df.shape)


(9499165, 11) (690666, 3)


In [9]:
# 4. drop reputation column / creationdateofowner

users_df = users_df.drop('Reputation', axis=1)
posts_df = posts_df.drop('CreationDateOfOwner', axis=1)
print(posts_df.shape, users_df.shape)


(9499165, 10) (690666, 2)


In [10]:
#5.1. Eliminate answers which do not have the question
#5.2. If a answer is faster than question of it, eliminate the answer

questions = posts_df[posts_df['PostTypeId'] == 1]
answers = posts_df[posts_df['PostTypeId'] == 2]
print(answers.shape)
print(questions.shape)


qna = answers.set_index('ParentId')\
        .join(questions, how='inner',\
              lsuffix='A', rsuffix='Q')
print(qna.shape)

qna = qna[qna['CreationDateA'] >= qna['CreationDateQ']]
qna = qna[['PostIdA']]
print(qna.shape)


(6396879, 10)
(3102286, 10)
(6152783, 19)
(6152120, 1)


In [11]:
#qna.info()

In [12]:
qna = qna.set_index('PostIdA')

answers = answers.join(qna, how='inner', lsuffix='', rsuffix='Q')
print(answers.shape)

posts_df = pd.concat([questions, answers])
posts_df = posts_df.set_index('PostId')
posts_df['PostId'] = posts_df.index

print(posts_df.shape, users_df.shape)
      
print(posts_df.info())

(6152120, 10)
(9254406, 10) (690666, 2)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 9254406 entries, 4 to 12918368
Data columns (total 10 columns):
PostTypeId          int32
AcceptedAnswerId    float32
CreationDate        datetime64[ns]
Score               float32
OwnerUserId         int32
AnswerCount         float32
CommentCount        float32
ParentId            float32
BodyWordNum         float64
PostId              int64
dtypes: datetime64[ns](1), float32(5), float64(1), int32(2), int64(1)
memory usage: 529.5 MB
None


In [13]:
users_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 690666 entries, -1 to 11963865
Data columns (total 2 columns):
CreationDate      690666 non-null datetime64[ns]
LastAccessDate    690666 non-null datetime64[ns]
dtypes: datetime64[ns](2)
memory usage: 15.8 MB


In [32]:
posts_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9254406 entries, 4 to 12918368
Data columns (total 10 columns):
PostTypeId          int32
AcceptedAnswerId    float32
CreationDate        datetime64[ns]
Score               float32
OwnerUserId         int32
AnswerCount         float32
CommentCount        float32
ParentId            float32
BodyWordNum         float64
PostId              int64
dtypes: datetime64[ns](1), float32(5), float64(1), int32(2), int64(1)
memory usage: 849.5 MB


In [126]:
# 6. verify churners
    
joined = users_df.join(posts_df.set_index('OwnerUserId') , how='left', rsuffix = 'P')
joined_int_group = joined[ (joined['CreationDateP'] >= (joined['CreationDate'] + pd.Timedelta(days=30))) & \
                            (joined['CreationDateP'] <= (joined['CreationDate'] + pd.Timedelta(days=210)))      
                          ]
joined_int_group = joined.groupby(joined.index).count()['PostId']

In [128]:
joined.info()
joined_int_group.describe()
joined_int_group[:3]

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8836365 entries, -1 to 11963865
Data columns (total 11 columns):
CreationDate        datetime64[ns]
LastAccessDate      datetime64[ns]
PostTypeId          float64
AcceptedAnswerId    float32
CreationDateP       datetime64[ns]
Score               float32
AnswerCount         float32
CommentCount        float32
ParentId            float32
BodyWordNum         float64
PostId              float64
dtypes: datetime64[ns](3), float32(5), float64(3)
memory usage: 640.5 MB


-1      0
 1    130
 2     11
Name: PostId, dtype: int64

In [168]:
import numpy as np

result_per_user = users_df.join(joined_int_group, how='inner')
result_per_user.rename(columns = {'PostId': 'countInInterval'}, inplace = True)
result_per_user['isChurn'] = np.where(result_per_user['countInInterval'] == 0, True, False)
result_per_user = result_per_user.drop('countInInterval', axis=1)

In [169]:
result_per_user.info()
result_per_user['isChurn'].describe()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 690666 entries, -1 to 11963865
Data columns (total 3 columns):
CreationDate      690666 non-null datetime64[ns]
LastAccessDate    690666 non-null datetime64[ns]
isChurn           690666 non-null bool
dtypes: bool(1), datetime64[ns](2)
memory usage: 16.5 MB


count     690666
unique         2
top        False
freq      567859
Name: isChurn, dtype: object

### statistics
- total 690666 people
- 17.8% is Churn (i.e. 1:4 ratio of 1/0 label)

- split train:val:test -> 6:2:2 


In [170]:
# 7. split

train_user = result_per_user.sample(frac=0.6,random_state=200)
val_test_user = result_per_user.drop(train_user.index)
val_user = val_test_user.sample(frac=0.5, random_state=321)
test_user = val_test_user.drop(val_user.index)

In [171]:
print(train_user['isChurn'].describe())
print(val_user['isChurn'].describe())
print(test_user['isChurn'].describe())

count     414400
unique         2
top        False
freq      340721
Name: isChurn, dtype: object
count     138133
unique         2
top        False
freq      113595
Name: isChurn, dtype: object
count     138133
unique         2
top        False
freq      113543
Name: isChurn, dtype: object


### remove all posts that are from val, test users

In [172]:
posts_by_val_users = posts_df.set_index('OwnerUserId').join(val_user, how='inner', lsuffix='P')['PostId']
posts_by_test_users = posts_df.set_index('OwnerUserId').join(test_user, how='inner', lsuffix='P')['PostId']

In [173]:
print(posts_by_val_users.describe())
print(posts_by_test_users.describe())

count    1.755770e+06
mean     5.672607e+06
std      3.227903e+06
min      6.000000e+00
25%      2.899012e+06
50%      5.647128e+06
75%      8.335888e+06
max      1.278398e+07
Name: PostId, dtype: float64
count    1.793890e+06
mean     5.762708e+06
std      3.211179e+06
min      2.500000e+01
25%      3.040644e+06
50%      5.757459e+06
75%      8.435578e+06
max      1.291837e+07
Name: PostId, dtype: float64


In [174]:
posts_df_hidden = posts_df.drop(posts_by_val_users.values)
posts_df_hidden = posts_df_hidden.drop(posts_by_test_users.values)

In [175]:
posts_df_hidden.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5704746 entries, 4 to 12918368
Data columns (total 10 columns):
PostTypeId          int32
AcceptedAnswerId    float32
CreationDate        datetime64[ns]
Score               float32
OwnerUserId         int32
AnswerCount         float32
CommentCount        float32
ParentId            float32
BodyWordNum         float64
PostId              int64
dtypes: datetime64[ns](1), float32(5), float64(1), int32(2), int64(1)
memory usage: 326.4 MB


### final statistics

- #of posts after refinement : 9254406
- #of users after refinement : 690666

- train:val:test split ratio : 6:2:2
- isChurn:notChurn ratio in all splits : approx. 1:4


- posts decrease due to hiding posts by val/test users : 9254406 -> 5704746 (38% deleted)

In [176]:
posts_df_hidden = posts_df_hidden.drop('PostId', axis=1)

In [177]:
print(train_user.info())
print(posts_df_hidden.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 414400 entries, 1083952 to 576689
Data columns (total 3 columns):
CreationDate      414400 non-null datetime64[ns]
LastAccessDate    414400 non-null datetime64[ns]
isChurn           414400 non-null bool
dtypes: bool(1), datetime64[ns](2)
memory usage: 9.9 MB
None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 5704746 entries, 4 to 12918368
Data columns (total 9 columns):
PostTypeId          int32
AcceptedAnswerId    float32
CreationDate        datetime64[ns]
Score               float32
OwnerUserId         int32
AnswerCount         float32
CommentCount        float32
ParentId            float32
BodyWordNum         float64
dtypes: datetime64[ns](1), float32(5), float64(1), int32(2)
memory usage: 282.9 MB
None


In [178]:
# output

OUTPUT_USERS_TRAIN = '../dataset_dist/users_train_dist.csv'
OUTPUT_USERS_VAL = '../dataset_dist/users_val_dist.csv'
OUTPUT_USERS_TEST = '../dataset_dist/users_test_dist.csv'

OUTPUT_POSTS = '../dataset_dist/posts_dist.csv'


train_user.to_csv(OUTPUT_USERS_TRAIN, mode='w', header=True, index=True) # index true!
val_user.to_csv(OUTPUT_USERS_VAL, mode='w', header=True, index=True)
test_user.to_csv(OUTPUT_USERS_TEST, mode='w', header=True, index=True)

posts_df_hidden.to_csv(OUTPUT_POSTS, mode='w', header=True, index=True) 